In [2]:
from getpass import getpass
openai_api_key = getpass()

In [3]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor, ConversationalAgent
from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory
from langchain import OpenAI, LLMChain, PromptTemplate
from langchain.utilities import GoogleSearchAPIWrapper
import langchain
langchain.debug = True

In [4]:
import os
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings


if not os.path.exists("../.doc_db/streamlit_chroma_db"):
    raise Exception("The Chroma database for Streamlit does not exist. Please run the script `doc_retriever.py` to create it.")

# load from disk
db = Chroma(persist_directory="../.doc_db/streamlit_chroma_db", embedding_function=OpenAIEmbeddings(openai_api_key=openai_api_key))
retriever = db.as_retriever()
retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs["fetch_k"] = 4
retriever.search_kwargs["maximal_marginal_relevance"] = True
retriever.search_kwargs["k"] = 4

chain_qa = RetrievalQA.from_chain_type(
    llm=OpenAI(temperature=0, max_tokens=2000, openai_api_key=openai_api_key), chain_type="stuff", retriever=retriever)

In [5]:
docs = db.similarity_search("How to get the camera input ?")
print(docs[0].page_content)

serde = CameraInputSerde()

        camera_input_state = register_widget(
            "camera_input",
            camera_input_proto,
            user_key=key,
            on_change_handler=on_change,
            args=args,
            kwargs=kwargs,
            deserializer=serde.deserialize,
            serializer=serde.serialize,
            ctx=ctx,
        )


In [6]:
from langchain.tools.python.tool import PythonREPLTool

tools = [
    Tool(
        name="Streamlit up to date source code",
        func=chain_qa.run,
        description="useful for when you need to answer questions about the streamlit Python API. Use this tool in priority. Input should be a fully formed question.",
    ),
    PythonREPLTool(),
]

In [7]:
prefix = """Assistant is a large language model trained by OpenAI. You are the Assistant.
Assistant is a chatbot specializing in python development (NOTHING ELSE) and you have a conversation with a human. Based on the conversation you have with this human and the new instructions or question he will input, you must update the python code that is compatible with python 3.9. Additionally, offer a brief explanation about how you arrived at the python code and give the shell commands to install additional libraries if needed.
If the input is a question, you must explain the code only and additionnaly propose some code. Do not halucinate or make up information. If you do not know the answer, just say "I don't know". If the question is not related to coding, just say "I not design to answer you about this question.".

The current python code you must update is the following:
```python
{python_code}
```
Use the streamlit built in functions in priority. If the user ask to use a specific library, you can use it. If he need to install the python module give the shell command to install it.
The code MUST be documented as much as possible and you MUST include the necessary imports.

Write your anwser in the following format:
```python
the code you generated
```
the explanation of the code you generated

If you did not generated any code (for instance when the user ask a question, not an instruction), this is the format:
```python
None
```
the anwser to the question

You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ConversationalAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["python_code", "input", "chat_history", "agent_scratchpad"],
)

In [8]:
memory = ConversationBufferMemory(memory_key="chat_history", input_key="input", output_key="output")
agent = ConversationalAgent.from_llm_and_tools(llm=OpenAI(temperature=0, openai_api_key=openai_api_key),
                                               prefix=prefix, suffix=suffix,
                                               tools=tools, verbose=True,
                                               input_variables=["python_code", "input", "chat_history", "agent_scratchpad"])
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory
)
memory.clear()

In [72]:
agent_chain.run(input="What is ChatGPT ?", python_code="import streamlit as st")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "What is ChatGPT ?",
  "python_code": "import streamlit as st",
  "chat_history": ""
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "What is ChatGPT ?",
  "python_code": "import streamlit as st",
  "chat_history": "",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "Assistant is a large language model trained by OpenAI. You are the Assistant.\nAssistant is a chatbot specializing in python development (NOTHING ELSE) and you have a conversation with a human. Based on the conversation you have with this human and the new instructions or question he will input, you must update the python code that is compatible with python 3.9. Additionally, offer a brief explanation about how you arrived at the 

'ChatGPT is a large language model trained by OpenAI. It is used to generate natural language responses to user input. It is based on the GPT-3 model and is designed to be used in chatbot applications.'

In [73]:
agent_chain.run(input="How can I add a button ?", python_code="import streamlit as st")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "How can I add a button ?",
  "python_code": "import streamlit as st",
  "chat_history": "Human: What is ChatGPT ?\nAI: ChatGPT is a large language model trained by OpenAI. It is used to generate natural language responses to user input. It is based on the GPT-3 model and is designed to be used in chatbot applications."
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "How can I add a button ?",
  "python_code": "import streamlit as st",
  "chat_history": "Human: What is ChatGPT ?\nAI: ChatGPT is a large language model trained by OpenAI. It is used to generate natural language responses to user input. It is based on the GPT-3 model and is designed to be used in chatbot applications.",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:OpenAI] Entering LL

Error in ConsoleCallbackHandler.on_retriever_start callback: retriever


[llm/end] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:OpenAI] [3.49s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Thought: Do I need to use a tool? Yes\nAction: Streamlit up to date source code\nAction Input: How to add a button in Streamlit",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 643,
      "completion_tokens": 34,
      "total_tokens": 677
    },
    "model_name": "text-davinci-003"
  },
  "run": null
}
[chain/end] [1:chain:AgentExecutor > 2:chain:LLMChain] [3.49s] Exiting Chain run with output:
{
  "text": "Thought: Do I need to use a tool? Yes\nAction: Streamlit up to date source code\nAction Input: How to add a button in Streamlit"
}
[tool/start] [1:chain:AgentExecutor > 4:tool:Streamlit up to date source code] Entering Tool run with input:
"How to add a button in Streamlit"
[chain/start] [1:chain

Error in ConsoleCallbackHandler.on_retriever_end callback: No retriever Run found to be traced


[chain/start] [1:chain:AgentExecutor > 4:tool:Streamlit up to date source code > 5:chain:RetrievalQA > 6:chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 4:tool:Streamlit up to date source code > 5:chain:RetrievalQA > 6:chain:MapReduceDocumentsChain > 7:chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "# Copyright (c) Streamlit Inc. (2018-2022) Snowflake Inc. (2022)\n#\n# Licensed under the Apache License, Version 2.0 (the \"License\");\n# you may not use this file except in compliance with the License.\n# You may obtain a copy of the License at\n#\n#     http://www.apache.org/licenses/LICENSE-2.0\n#\n# Unless required by applicable law or agreed to in writing, software\n# distributed under the License is distributed on an \"AS IS\" BASIS,",
      "question": "How to add a button in Streamlit"
    },
    {
      "context": "# Copyright (c) Streamlit Inc. (2018-2022) Snowflake Inc. (202

"I'm sorry, I don't know how to add a button in Streamlit."

In [9]:
agent_chain.run(input="I want to get the camera input in the streamlit app", python_code="import streamlit as st")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "I want to get the camera input in the streamlit app",
  "python_code": "import streamlit as st",
  "chat_history": ""
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "I want to get the camera input in the streamlit app",
  "python_code": "import streamlit as st",
  "chat_history": "",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "Assistant is a large language model trained by OpenAI. You are the Assistant.\nAssistant is a chatbot specializing in python development (NOTHING ELSE) and you have a conversation with a human. Based on the conversation you have with this human and the new instructions or question he will input, you must update the python code that is compatible with python 3.9. A

'You can use the streamlit.camera() function to get the camera input in the streamlit app. You can find more information about this function in the Streamlit up to date source code. Additionally, you will need to install the opencv-python module. To do so, you can use the following shell command: pip install opencv-python.'